# Library

In [31]:
import pandas as pd
import numpy as np
import seaborn as sns
import tldextract
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.feature_extraction import FeatureHasher
import tensorflow as tf
from tensorflow.keras.layers import Embedding, Input, Dense, Flatten
from tensorflow.keras.models import Model
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm

# Read Data

In [32]:
PATH = r"C:\Users\dommy\OneDrive\Documenti\GSoC - Final\GSoC\CSV\threads.csv"

In [33]:
df = pd.read_csv(PATH)
df

,id,site_id,board_id,creator,creator_id,label,name,url,created_on,last_post_on,db_created_on,db_updated_on
0,1098607808,1019,258,randalthor,408,NaN,where 2 now,http://dreadditevelidot.onion/post/d0e38d2ba74...,2020-01-09 15:35:04.078374,NaN,2020-01-09 15:35:04.078374,2020-01-09 15:35:04.078374
1,1208831554,1019,18,sickde666,7360,NaN,Login,http://dreadditevelidot.onion/post/6ada35da0b3...,2020-01-08 14:32:21.56381,NaN,2020-01-08 14:32:21.56381,2020-01-08 14:32:21.56381
2,1731938282,1019,56,straightdave,13667,NaN,Withdrawals locked,http://dreadditevelidot.onion/post/17e952b5b6d...,2019-08-07 23:48:54.215944,NaN,2019-08-07 23:48:54.215944,2019-08-07 23:48:54.215944
3,1161153828,1019,56,Krispykremelad,4853,NaN,No moderator on my dispute for 8 days?,http://dreadditevelidot.onion/post/9f2dad3183b...,2019-08-07 23:48:57.902027,NaN,2019-08-07 23:48:57.902027,2019-08-07 23:48:57.902027
4,929896129,1019,56,yungtiller,5524,NaN,No pin,http://dreadditevelidot.onion/post/5f2a5a75a93...,2019-08-07 23:49:01.210173,NaN,2019-08-07 23:49:01.210173,2019-08-07 23:49:01.210173
...,...,...,...,...,...,...,...,...,...,...,...,...
75117,566787583,1019,441,fairfax,33131,NaN,Caramel Haze (2-5€ per gram) GER -> WW,http://dreadditevelidot.onion/post/20a8465e629...,2020-01-09 21:07:16.622158,NaN,2020-01-09 21:07:16.622158,2020-01-09 21:07:16.622158
75118,161231059,1019,441,Denode,41474,NaN,HQ Sniffed US CC 95% Valid DENODE Team,http://dreadditevelidot.onion/post/0c17b7f0db4...,2020-01-09 21:07:18.79134,NaN,2020-01-09 21:07:18.79134,2020-01-09 21:07:18.79134
75119,26637282,1019,441,Denode,41474,NaN,Fresh CANADA Random Fullz DENODE,http://dreadditevelidot.onion/post/c974b59d52f...,2020-01-09 21:07:21.14179,NaN,2020-01-09 21:07:21.14179,2020-01-09 21:07:21.14179
75120,341142936,1019,441,DomesticProduct,42899,NaN,"New US vendor: DomesticProduct (K, MDMA, & more!)",http://dreadditevelidot.onion/post/eb59e476ade...,2020-01-09 21:07:23.72459,NaN,2020-01-09 21:07:23.72459,2020-01-09 21:07:23.72459


In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75122 entries, 0 to 75121
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             75122 non-null  int64  
 1   site_id        75122 non-null  int64  
 2   board_id       75122 non-null  int64  
 3   creator        74867 non-null  object 
 4   creator_id     75122 non-null  int64  
 5   label          0 non-null      float64
 6   name           75122 non-null  object 
 7   url            75122 non-null  object 
 8   created_on     75122 non-null  object 
 9   last_post_on   0 non-null      float64
 10  db_created_on  75122 non-null  object 
 11  db_updated_on  75122 non-null  object 
dtypes: float64(2), int64(4), object(6)
memory usage: 6.9+ MB


In [35]:
df.describe()

,id,site_id,board_id,creator_id,label,last_post_on
count,7.512200e+04,75122.0,75122.000000,75122.000000,0.0,0.0
mean,1.075652e+09,1019.0,63.129749,11827.177005,NaN,NaN
std,6.199184e+08,0.0,97.533545,12627.206364,NaN,NaN
min,3.372500e+04,1019.0,1.000000,-1.000000,NaN,NaN
25%,5.369132e+08,1019.0,10.000000,1969.000000,NaN,NaN
50%,1.080713e+09,1019.0,18.000000,7214.000000,NaN,NaN
75%,1.610658e+09,1019.0,65.000000,16937.750000,NaN,NaN
max,2.147418e+09,1019.0,446.000000,47346.000000,NaN,NaN


In [36]:
df.describe(include='object')

,creator,name,url,created_on,db_created_on,db_updated_on
count,74867,75122,75122,75122,75122,75122
unique,24062,69833,75122,75122,75122,75122
top,[deleted],MEDICAL GRADE LIQUID KETAMINE AVAILABLE. USA a...,http://dreadditevelidot.onion/post/d0e38d2ba74...,2020-01-09 15:35:04.078374,2020-01-09 15:35:04.078374,2020-01-09 15:35:04.078374
freq,807,54,1,1,1,1


In [37]:
df.nunique()

id               75122
site_id              1
board_id           382
creator          24062
creator_id       24428
label                0
name             69833
url              75122
created_on       75122
last_post_on         0
db_created_on    75122
db_updated_on    75122
dtype: int64

In [38]:
df.isnull().sum()

id                   0
site_id              0
board_id             0
creator            255
creator_id           0
label            75122
name                 0
url                  0
created_on           0
last_post_on     75122
db_created_on        0
db_updated_on        0
dtype: int64

In [39]:
df[(df['db_created_on'] == df['db_updated_on']) & (df['db_updated_on'] == df['created_on'])][['db_created_on', 'db_updated_on', 'created_on']]

,db_created_on,db_updated_on,created_on
0,2020-01-09 15:35:04.078374,2020-01-09 15:35:04.078374,2020-01-09 15:35:04.078374
1,2020-01-08 14:32:21.56381,2020-01-08 14:32:21.56381,2020-01-08 14:32:21.56381
2,2019-08-07 23:48:54.215944,2019-08-07 23:48:54.215944,2019-08-07 23:48:54.215944
3,2019-08-07 23:48:57.902027,2019-08-07 23:48:57.902027,2019-08-07 23:48:57.902027
4,2019-08-07 23:49:01.210173,2019-08-07 23:49:01.210173,2019-08-07 23:49:01.210173
...,...,...,...
75117,2020-01-09 21:07:16.622158,2020-01-09 21:07:16.622158,2020-01-09 21:07:16.622158
75118,2020-01-09 21:07:18.79134,2020-01-09 21:07:18.79134,2020-01-09 21:07:18.79134
75119,2020-01-09 21:07:21.14179,2020-01-09 21:07:21.14179,2020-01-09 21:07:21.14179
75120,2020-01-09 21:07:23.72459,2020-01-09 21:07:23.72459,2020-01-09 21:07:23.72459


In [40]:
df.drop(columns=['site_id', 'label', 'last_post_on', 'db_created_on', 'db_updated_on'], inplace=True)

In [41]:
df.head()

,id,board_id,creator,creator_id,name,url,created_on
0,1098607808,258,randalthor,408,where 2 now,http://dreadditevelidot.onion/post/d0e38d2ba74...,2020-01-09 15:35:04.078374
1,1208831554,18,sickde666,7360,Login,http://dreadditevelidot.onion/post/6ada35da0b3...,2020-01-08 14:32:21.56381
2,1731938282,56,straightdave,13667,Withdrawals locked,http://dreadditevelidot.onion/post/17e952b5b6d...,2019-08-07 23:48:54.215944
3,1161153828,56,Krispykremelad,4853,No moderator on my dispute for 8 days?,http://dreadditevelidot.onion/post/9f2dad3183b...,2019-08-07 23:48:57.902027
4,929896129,56,yungtiller,5524,No pin,http://dreadditevelidot.onion/post/5f2a5a75a93...,2019-08-07 23:49:01.210173


# Data Wrangling

## Created_on feature

In [42]:
df['created_on'] = pd.to_datetime(df['created_on'], format='%Y-%m-%d %H:%M:%S.%f').dt.strftime('%Y-%m-%d')
df.head()

,id,board_id,creator,creator_id,name,url,created_on
0,1098607808,258,randalthor,408,where 2 now,http://dreadditevelidot.onion/post/d0e38d2ba74...,2020-01-09
1,1208831554,18,sickde666,7360,Login,http://dreadditevelidot.onion/post/6ada35da0b3...,2020-01-08
2,1731938282,56,straightdave,13667,Withdrawals locked,http://dreadditevelidot.onion/post/17e952b5b6d...,2019-08-07
3,1161153828,56,Krispykremelad,4853,No moderator on my dispute for 8 days?,http://dreadditevelidot.onion/post/9f2dad3183b...,2019-08-07
4,929896129,56,yungtiller,5524,No pin,http://dreadditevelidot.onion/post/5f2a5a75a93...,2019-08-07


In [43]:
df['created_on'].value_counts().sort_index(ascending=False)

created_on
2020-04-15        5
2020-04-14        2
2020-04-13        1
2020-02-23      173
2020-02-22      119
2020-02-21        8
2020-01-09    14789
2020-01-08    25483
2020-01-07    18092
2020-01-06      106
2020-01-01       63
2019-12-31       98
2019-12-30        7
2019-11-26       66
2019-11-11     1022
2019-11-06      441
2019-11-05     1489
2019-11-04       89
2019-10-30        4
2019-10-29      104
2019-10-28       37
2019-10-25        9
2019-10-24      111
2019-08-09       44
2019-08-08     4656
2019-08-07     1719
2019-08-06     6385
Name: count, dtype: int64